# Initial Ideas for evaporator code

Here's some simple ideas for how best to code an evaporating body

#### Physical approach

Let's think about the real physics going on, and see if we can adapt known models to account for that.

The cause of the signal in evaporator transits is not the planet, but the dust cloud.

To me, this dust cloud has three possibilities:
 * Spherical cloud around planet which decreases density with distance
 * Smeared tail behind planet which decreases density with distance
 * Smeared tail in front of planet which decreases density with distance
 
In all three cases, we could iteractively fit a Mandel+Agol transit model to account for the change in size over time. e.g:
 
 <img src="MandelAgolIdeas.png">
 
 This is not quite what happens, because there's thinks like Roche lobe overflow at L1/L2, but maybe it's enough for a general model?

Alternatively, and possibly more realistically, these dust grains are pushed onto a new orbit, so potentially we should include a Keplerian signal for that which should include for curvature in the tail.

In any case, we would then have a couple of hyper parameters for each transit governing:
* the direction of motion of the dust cloud w.r.t. the planet (+ being leading, 0 being sphere, - being trailing)
* the rate of dispersion over time
* total dust production (which should governe depth)

In this sort of model, the dust would be conserved - i.e. the density of the dust between transit model iterations would be proportional to the change in area (i.e. dispersion)

##### Forward-scattering

 <img src="K222_model.jpg">

In the previous models (e.g. with Kepler), they have included a term to model the effect that this tail of dust is actually *scattering* (ie not just blocking) the light. That means less light in transit, but also extra light around transit due to the fact that the tail is directing light back to the observer.

Personally I think this is too much to include in the model because: 
* 3D reflection geometry is non-trivial
* It depends on both dust grain size & separation - two unknowns
* The amount of scattering will certainly be less with TESS because of the redder wavelength

But maybe we can include some sort of density-dependent boost effect around central transit with a couple of floating parameters:
* dust density : boost height ratio (for the height of the Gaussian)
* reflection angle (for the width of the Gaussian)

##### Pseudo code:

Parameters
* planet period = $P$
* planet transit time = $t_0$
* planet impact param = $b$
* either:
    * dust production mean = $D_\mu$
    * dust production std = $D_\sigma$
* or:
    * SHM kernel for dust production in celerite with parameters mean $D_\mu$ and lengthscale $l$.
* dust speed = $v$
* dust dispersion = $\delta$
* density to reflection boost = $b_D$
* reflection angle = $\Theta_b$

For each transit:
* draw dust production ($D_i$) from the $D_\mu$ and $D_\sigma$ distributions
* Create some number of steps to iterate over from -pi/2 to 0 in phase producing phantom planets at each

For each step:
* Using $v$ and the orbital velocity of the planet from ($P$) we generate an eccentric orbit for the M+A model. The new orbit is either at aphelion (for trailing tail) or perihelion (leading tail) at the 
* We then compute the new "phantom planet" orbit given $P$, $v$, $t_l$, $t_c$
* Depending on the $\delta$ and $t_l$ we calculate a new phantom planet radius, and a new "density" (e.g. alpha) for the transit.

Then, for each transit:
* Sum all the transit lightcurves for the phantom planets.
* Use $b_D$ and $\Theta_b$ to add a reflection boost

In [ ]:
def EvapModel(t, paramdic, use_gp=True, n_steps=10):
    #Get all transits 
    phase=(t-paramdic['t_0'])/paramdic['P']
    trns_numbers = np.array([nt for nt in np.arange(np.floor(phase[0]),np.ceil(phase[-1]),1.0) if np.min(abs(phase-nt))<0.5*paramdic['P']])
    trns_times   = paramdic['t_0']+paramdic['P']*trns_numbers
    
    if use_gp:
        #Getting dust production from a GP
    else:
        #Getting dust production from the input mean/std
    for n_tr in trns_numbers:
        
        # let's build our steps as a numpy matrix
        # Initial positions of the orbits
        init_positions=np.linspace(-0.5*np.pi,0,n_steps)
        